In [64]:
example = """
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
""".strip().splitlines()

with open("day04.txt", "r") as f:
    data = [line.strip() for line in f.readlines()]


In [69]:
from typing import Dict, Iterable, Union, List
import re

class Passport(object):
    fields = {
        # "cid": int # Temporarilly allow north-pole credentials
        "pid": lambda f: re.match("^[0-9]{9}$", f),
        "byr": lambda f: 1920 <= int(f) <= 2002,
        "iyr": lambda f: 2010 <= int(f) <= 2020,
        "eyr": lambda f: 2020 <= int(f) <= 2030,
        "hgt": lambda f: re.match("^1([5-8]\d|9[0-3])cm$|^(59|6\d|7[0-6])in$", f),
        "hcl": lambda f: re.match("^#[0-9a-f]{6}$", f),
        "ecl": lambda f: re.match("^(amb|blu|brn|gry|grn|hzl|oth)$", f),
    }

    def __init__(self, **fields: Dict[str, str]):
        self._fields = fields
        
    def is_valid(self, data_validation: bool = False) -> bool:
        for key, validate in Passport.fields.items():
            if key not in self._fields:
                return False

            if data_validation:
                try:
                    if not validate(self._fields[key]):
                        return False
                except Exception:
                    return False
            
        return True

    def __getattr__(self, key: str) -> Union[str, int, None]:
        if key in self._fields:
            return self._fields[key]
        else:
            return None

    def __str__(self) -> str:
        return str(self._fields)

    
    def __repr__(self) -> str:
        return f"Passport(**{self._fields})"
        
    @staticmethod
    def from_str(input: str) -> "Passport":
        fields = {
            entry[0]: entry[1]
            for entry in re.findall(r'(\w+):([^\s]+)', input, re.MULTILINE)
        }

        return Passport(**fields)

    def from_lines(input: List[str]) -> Iterable["Passport"]:
        parts = []
        for line in input:
            if line == "":
                yield Passport.from_str(" ".join(parts))
                parts = []
            else:
                parts.append(line)

        yield Passport.from_str("\n".join(parts))


len(list(p for p in Passport.from_lines(example) if p.is_valid()))

2

In [66]:

assert Passport.fields["byr"]("2002")
assert not Passport.fields["byr"]("2003")

assert Passport.fields["hgt"]("60in")
assert Passport.fields["hgt"]("190cm")
assert not Passport.fields["hgt"]("190in")
assert not Passport.fields["hgt"]("190")

assert Passport.fields["hcl"]("#123abc")
assert not Passport.fields["hcl"]("#123abz")
assert not Passport.fields["hcl"]("123abc")

assert Passport.fields["ecl"]("brn")
assert not Passport.fields["ecl"]("wat")

assert Passport.fields["pid"]("000000001")
assert not Passport.fields["pid"]("0123456789")


In [70]:
invalid = """
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
""".strip().splitlines()

assert len(list(p for p in Passport.from_lines(invalid) if p.is_valid(data_validation=True))) == 0

In [71]:

valid = """
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
""".strip().splitlines()

assert list(p for p in Passport.from_lines(valid) if not p.is_valid(data_validation=True)) == []

In [73]:
len(list(p for p in Passport.from_lines(data) if p.is_valid(data_validation=False)))

202

In [74]:
len(list(p for p in Passport.from_lines(data) if p.is_valid(data_validation=True)))

137